In [1]:
import cv2
import numpy as np

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

from keras.preprocessing import image
import os 

Using TensorFlow backend.


In [2]:
# Create the model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [6]:
faceCascade = cv2.CascadeClassifier("Resources/cascade/haarcascade_frontalface_default.xml")

model.load_weights('Resources/Trained_Models/model.h5')

# dictionary which assigns each label an emotion (alphabetical order)
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

modes = {0: "webcam", 1: "image"}
mode = modes[0]
saveDir = "Resources\images"

In [7]:
if mode == "webcam":
    cap = cv2.VideoCapture(0)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(gray, scaleFactor = 1.3, minNeighbors = 5)
        
        for (x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y-50),(x+w,y+h+10),(255,0,0),thickness=2)
            roi_gray=gray[y:y+w,x:x+h]
            roi_gray=cv2.resize(roi_gray,(48,48))
            img_pixels = image.img_to_array(roi_gray)
            img_pixels = np.expand_dims(img_pixels, axis = 0)
            img_pixels /= 255

            prediction = model.predict(img_pixels)

            # Print percentage
            for i in range(len(emotion_dict)):
                txt = "{}: {:.2f}%".format(emotion_dict[i], prediction[0][i]*100)
                cv2.putText(frame, txt, (0,20 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            maxindex = int(np.argmax(prediction))
            cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        resized_img = cv2.resize(frame, (640, 480))
        cv2.imshow('Facial emotion analysis ',resized_img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        if cv2.waitKey(1) & 0xFF == ord('s'):
            os.chdir(saveDir)
            cv2.imwrite("image.png", resized_img) 
        
    cap.release()
    cv2.destroyAllWindows()

In [5]:
if mode== "image":
    img = cv2.imread("Resources/images/test.png")

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),thickness=2)
        roi_gray=gray[y:y+w,x:x+h]
        roi_gray=cv2.resize(roi_gray,(48,48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255

        prediction = model.predict(img_pixels)

        # Print percentage
#         for i in range(len(emotion_dict)):
#             txt = "{}: {:.2f}%".format(emotion_dict[i], prediction[0][i]*100)
#             cv2.putText(img, txt, (0,20 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

        maxindex = int(np.argmax(prediction))
        cv2.putText(img, emotion_dict[maxindex], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2, cv2.LINE_AA)
        
    resized_img = cv2.resize(img, (1000, 700))
    cv2.imshow('Facial emotion analysis ',resized_img)
    
    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        elif cv2.waitKey(1) & 0xFF == ord('s'):
            os.chdir(saveDir)
            cv2.imwrite("image.png", resized_img) 
        
    cv2.destroyAllWindows()